# Django basics (na podstawie `django_examples/hello_django`)

Ten notebook to krótkie, praktyczne wprowadzenie do Django, oparte o projekt z katalogu `django_examples/hello_django`.

## Czym jest Django?

- Framework webowy w Pythonie typu *batteries‑included*.
- Daje gotowe mechanizmy: routing, ORM, szablony, admin, autoryzacja.
- Typowy podział: **projekt** (settings/urls) + **aplikacje** (logika domenowa).

## Struktura projektu w tym repo

Najważniejsze elementy:

- `django_examples/hello_django/manage.py` – główny punkt wejścia do komend Django.
- `django_examples/hello_django/hello_django/` – konfiguracja projektu:
  - `settings.py` – ustawienia (np. `INSTALLED_APPS`, baza danych).
  - `urls.py` – główny routing.
  - `wsgi.py`/`asgi.py` – uruchamianie aplikacji.
- Aplikacje:
  - `simplerouting/` – proste widoki i routing.
  - `matematyka/` – kalkulator + zapis historii w bazie.
- `db.sqlite3` – lokalna baza danych (SQLite).

## Routing (URLConf)

Główne `urls.py` w projekcie dołącza routingi aplikacji:

```python
urlpatterns = [
    path('admin/', admin.site.urls),
    path("", include("simplerouting.urls")),
    path("matematyka/", include("matematyka.urls")),
]
```

Przykładowe URL-e z aplikacji:

- `/` → `simplerouting.views.index`
- `/hello/` → `simplerouting.views.hello`
- `/hello/<username>` → `simplerouting.views.hello`
- `/matematyka/<op>/<a>/<b>/` → `matematyka.views.calculate`

## Widoki (views)

Przykład prostego widoku zwracającego `HttpResponse`:

```python
# simplerouting/views.py

def hello(request, username=""):
    if username:
        text = f"Hello {username}"
        HelloUserNameHistory(username=username).save()
    else:
        text = "Hello World"
    return HttpResponse(text)
```

Przykład widoku renderującego szablon:

```python
# matematyka/views.py

def calculate(request, op, a, b):
    result = MathService.calculate(op, int(a), int(b))
    models.CalculationHistory.objects.create(...)
    return render(request, "matematyka/result.html", {"result": result})
```

## Modele i ORM

Django posiada wbudowany ORM. W tym projekcie są np.:

- `HelloUserNameHistory` w `simplerouting/models.py` – loguje użycia endpointu `/hello/<username>`.
- `CalculationHistory` w `matematyka/models.py` – zapisuje historię operacji kalkulatora.

Po zmianach modeli uruchamiasz migracje:

```bash
python manage.py makemigrations
python manage.py migrate
```

## Warstwa usług (services)

W aplikacji `matematyka` logika obliczeń jest wyciągnięta do osobnego modułu:

```python
# matematyka/services.py

class MathService:
    @staticmethod
    def calculate(op, a, b):
        return oprations[op](a, b)
```

To dobry wzorzec: widok jest cienki, a logika biznesowa żyje osobno.

## Konfiguracja w `settings.py`

Najważniejsze fragmenty:

- `INSTALLED_APPS` – dodane aplikacje: `matematyka`, `simplerouting`.
- `DATABASES` – SQLite w pliku `db.sqlite3`.
- `TEMPLATES` – domyślna konfiguracja szablonów (APP_DIRS=True).

## Najczęstsze komendy

```bash
# instalacja
pip install django

# uruchomienie serwera dev
python manage.py runserver

# tworzenie aplikacji
python manage.py startapp nazwa_aplikacji

# migracje
python manage.py makemigrations
python manage.py migrate

# panel admina
python manage.py createsuperuser
```

## Mini‑ćwiczenia (na bazie projektu)

1. Dodaj nowy endpoint w `simplerouting`, który przyjmie `username` i zapisze historię.
2. W `matematyka` dopisz obsługę dzielenia z kontrolą dzielenia przez zero.
3. Dodaj szablon `matematyka/result.html`, który estetycznie pokaże wynik.